In [ ]:
import pandas as pd
import pingouin as pg
import lightgbm as lgb
import shap
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt


plt.rcParams["font.family"] = "Malgun Gothic"  

df = pd.read_csv("preprocessing_filtered.csv")
df

ImportError: Numba needs NumPy 2.2 or less. Got NumPy 2.3.

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22308 entries, 0 to 22307
Data columns (total 45 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unnamed: 0                    22308 non-null  int64  
 1   neighbourhood_cleansed        22308 non-null  object 
 2   neighbourhood_group_cleansed  22308 non-null  object 
 3   latitude                      22308 non-null  float64
 4   longitude                     22308 non-null  float64
 5   price                         22308 non-null  float64
 6   estimated_occupancy_l365d     22308 non-null  int64  
 7   estimated_revenue_l365d       22308 non-null  float64
 8   availability_60               22308 non-null  int64  
 9   availability_365              22308 non-null  int64  
 10  instant_bookable              22308 non-null  int64  
 11  is_long_term                  22308 non-null  int64  
 12  is_activate                   22308 non-null  int64  
 13  a

In [4]:
df.columns

Index(['Unnamed: 0', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed',
       'latitude', 'longitude', 'price', 'estimated_occupancy_l365d',
       'estimated_revenue_l365d', 'availability_60', 'availability_365',
       'instant_bookable', 'is_long_term', 'is_activate', 'amenities',
       'amenities_cnt', 'room_type', 'structure_type', 'structure_category',
       'accommodates', 'bath_score_mul', 'beds', 'number_of_reviews',
       'number_of_reviews_ltm', 'number_of_reviews_l30d',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'id', 'host_id', 'name_length_group',
       'description_length_group', 'neighborhood_overview_exists',
       'host_about_length_group', 'host_is_superhost', 'host_has_profile_pic',
       'host_identity_verified', 'host_response_time_score',
       'host_response_rate_score', 'host_acce

In [36]:
# price 정규성 검정
price_data = df['price'].dropna()

stat, p = stats.shapiro(price_data)

print(f"Shapiro-Wilk 통계량: {stat:.4f}") #실제 데이터가 정규분포에 얼마나 가까운지를 수치화한 값 (0~1 사이)
print(f"p-value: {p:.4f}")
if p > 0.05:
    print("정규성을 가정할 수 없다")
else:
    print("정규성을 가정할 수 없다")

Shapiro-Wilk 통계량: 0.2379
p-value: 0.0000
정규성을 가정할 수 없다


In [37]:
#로그변환 

log_price = np.log1p(df['price'].dropna())  # log(1 + price)로 0도 처리 가능
stat, p = stats.shapiro(log_price)

print(f"Shapiro-Wilk 통계량: {stat:.4f}")
print(f"p-value: {p:.4f}")

if p > 0.05:
    print("로그 변환 후 정규성을 가정할 수 있다")
else:
    print("로그 변환 후에도 정규성을 가정할 수 없다")

Shapiro-Wilk 통계량: 0.9811
p-value: 0.0000
로그 변환 후에도 정규성을 가정할 수 없다


In [ ]:
from scipy.stats import mannwhitneyu

group1 = df[df['host_is_superhost'] == 1]['price'].dropna()
group0 = df[df['host_is_superhost'] == 0]['price'].dropna()

stat, p = mannwhitneyu(group1, group0, alternative='two-sided')
print(f"Mann-Whitney U 통계량: {stat:.4f}, p-value: {p:.4f}")

# 슈퍼호스트인지 아닌지 두그룹의 가격 분포에 차이가 있다

Mann-Whitney U 통계량: 45228149.5000, p-value: 0.0000


In [ ]:
superhost_median = df[df['host_is_superhost'] == 1]['price'].median()
non_superhost_median = df[df['host_is_superhost'] == 0]['price'].median()

print(f"슈퍼호스트 중앙값: {superhost_median}")
print(f"일반 호스트 중앙값: {non_superhost_median}")

# 슈퍼호스트 중앙값이 더 낮다. 합리적인 가격이 슈퍼호스트가 되기에 유리하다 

슈퍼호스트 중앙값: 129.0
일반 호스트 중앙값: 147.0


In [ ]:
binary_vars = ['host_identity_verified', 'host_location_boolean', 'host_location_ny',
    'neighborhood_overview_exists', 'is_long_term', 'instant_bookable', 'is_activate']
continuous_vars = [
    'price', 'estimated_occupancy_l365d', 'estimated_revenue_l365d',
    'availability_60', 'availability_365', 'amenities_cnt',
    'bath_score_mul', 'beds', 'accommodates', 'number_of_reviews',
    'number_of_reviews_ltm', 'number_of_reviews_l30d',
    'review_scores_rating', 'review_scores_accuracy',
    'review_scores_cleanliness', 'review_scores_checkin',
    'review_scores_communication', 'review_scores_location',
    'review_scores_value', 'host_response_time_score',
    'host_response_rate_score', 'host_acceptance_rate_score'
]

In [46]:
from scipy import stats
import pandas as pd
import numpy as np

# 결과 저장용 리스트
results = []

# 1. 수치형 변수 분석
for col in continuous_vars:
    group1 = data[data['host_is_superhost'] == 1][col].dropna()
    group0 = data[data['host_is_superhost'] == 0][col].dropna()

    # 정규성 검정 (Shapiro)
    p_norm1 = stats.shapiro(group1)[1] if len(group1) < 5000 else np.nan  # 샘플 수가 5000 이상이면 오류
    p_norm0 = stats.shapiro(group0)[1] if len(group0) < 5000 else np.nan

    # 둘 다 정규 → t-test, 아니면 Mann-Whitney
    if (p_norm1 > 0.05) and (p_norm0 > 0.05):
        stat, p_val = stats.ttest_ind(group1, group0)
        test_type = 't-test'
    else:
        stat, p_val = stats.mannwhitneyu(group1, group0, alternative='two-sided')
        test_type = 'Mann-Whitney'

    results.append({
        '변수': col,
        '유형': '수치형',
        '정규성_p(슈퍼호스트)': round(p_norm1, 4) if not np.isnan(p_norm1) else 'n/a',
        '정규성_p(일반호스트)': round(p_norm0, 4) if not np.isnan(p_norm0) else 'n/a',
        '검정 방법': test_type,
        'p-value': round(p_val, 4),
        '해석': '✅ 유의미한 차이' if p_val < 0.05 else '❌ 차이 없음'
    })

# 2. 이진형 변수 → 카이제곱
for col in binary_vars:
    contingency = pd.crosstab(data[col], data['host_is_superhost'])
    chi2, p_val, dof, ex = stats.chi2_contingency(contingency)

    results.append({
        '변수': col,
        '유형': '이진형',
        '정규성_p(슈퍼호스트)': '-',
        '정규성_p(일반호스트)': '-',
        '검정 방법': 'Chi-square',
        'p-value': round(p_val, 4),
        '해석': '✅ 유의미한 관계' if p_val < 0.05 else '❌ 관계 없음'
    })

# 결과를 데이터프레임으로 정리
results_df = pd.DataFrame(results)
results_df.sort_values('p-value', inplace=True)
results_df.reset_index(drop=True, inplace=True)
results_df.head(10)  # 상위 10개만 보기


NameError: name 'continuous_vars' is not defined

In [ ]:
sig_num  = t_df[t_df.p < 0.05]['variable'].tolist()
sig_cat  = chi_df[chi_df.p < 0.05]['variable'].tolist()
sig_vars = sig_num + sig_cat
print("유의미한 변수:", sig_vars)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

X = df[sig_vars + [TARGET]].copy()
y = X.pop(TARGET).astype(int)

cat_cols = X.select_dtypes(include=['object','category','bool']).columns
num_cols = X.select_dtypes(include=['int64','float64']).columns

preprocessor = ColumnTransformer([
    ('num', 'passthrough', num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

X_enc = preprocessor.fit_transform(X)
print("변환 후 변수 개수:", X_enc.shape[1])


In [ ]:
strategy_vars = [
    'host_has_profile_pic', 'host_identity_verified',
    'host_response_time_score', 'host_response_rate_score', 'host_acceptance_rate_score',
    'is_activate', 'instant_bookable', 'amenities_cnt',
    'name_length_group', 'description_length_group', 'neighborhood_overview_exists',
    'room_type', 'structure_type', 'structure_category',
    'accommodates', 'beds', 'bath_score_mul'
]
strategy_vars = [c for c in strategy_vars if c in df.columns]

Xs = df[strategy_vars + [TARGET]].copy()
ys = Xs.pop(TARGET).astype(int)

cat_s = Xs.select_dtypes(include=['object','category','bool']).columns
num_s = Xs.select_dtypes(include=['int64','float64']).columns

prep_s = ColumnTransformer([
    ('num', 'passthrough', num_s),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_s)
])

Xs_enc = prep_s.fit_transform(Xs)

X_tr, X_te, y_tr, y_te = train_test_split(
    Xs_enc, ys, test_size=0.2, random_state=42, stratify=ys)

clf_s = lgb.LGBMClassifier(
    n_estimators=300, learning_rate=0.05, class_weight='balanced', random_state=42)
clf_s.fit(X_tr, y_tr,
          eval_set=[(X_te, y_te)],
          early_stopping_rounds=50,
          verbose=False)

print("=== 전략 모델 성능 ===")
print(classification_report(y_te, clf_s.predict(X_te)))

explainer_s = shap.TreeExplainer(clf_s)
shap_values_s = explainer_s.shap_values(X_te)
shap.summary_plot(shap_values_s, X_te, feature_names=prep_s.get_feature_names_out())